In [5]:
import os
import sys
import time
import wget

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

print "Opening Trip Advisor"

driver = webdriver.Firefox(executable_path=r'C:\Program Files\geckodriver.exe')

location = raw_input("Enter the destination Location ")
current  = raw_input("Enter current location ")
#start_date = raw_input("Start Date")
#end_date = raw_input("End Date")

driver.get("https://www.tripadvisor.in/")
driver.find_element_by_xpath("//input[@tabindex='1']").clear()
driver.find_element_by_xpath("//input[@tabindex='1']").send_keys(location)
time.sleep(3)
# driver.find_element_by_xpath("//input[@tabindex='1']").send_keys(Keys.ENTER)
driver.find_element_by_xpath("//span[@class='picker-label target ui_picker_arrow_target']").click()
driver.find_element_by_xpath("//span[@data-date='2017-10-20']").click()
driver.find_element_by_xpath("//span[@data-date='2017-10-25']").click()
time.sleep(2)
driver.find_element_by_xpath("//button[@tabindex='5']").click()

time.sleep(3)


Opening Trip Advisor
Enter the destination Location Mumbai
Enter current location Delhi


In [6]:
print "Extracting Hotels Data"

src = driver.page_source

soup = BeautifulSoup(src, "lxml")

page_numbers = soup.find("pageNumbers")

page_numbers = soup.find("div",{"class","pageNumbers"})
page_numbers.find_all('a')
last = page_numbers.find_all('a').pop()
size = int(last['data-page-number'])

hotel_names = []
hotel_links = []
hotel_price = []

size = 2 #remove this after demo

for j in range(1,size):
    tiles = soup(attrs={'class': 'prw_rup prw_meta_hsx_three_col_listing'})
    
    prices = soup(attrs={'class': 'priceBlock'}) 

    for i in range(0, len(tiles)):
        hotel_names.append(tiles[i].find_all("div",{"class","listing_title"})[0].find_all("a")[-1].text)    
        hotel_links.append(tiles[i].find_all("div",{"class","listing_title"})[0].a['href'])
        if len(tiles[i].find_all("div",{"class","priceBlock"})) > 0 :
            if str(tiles[i].find_all("div",{"class","priceBlock"})[0].find_all("div",{"class","price"})[0].text[2:]) == "" :
                hotel_price.append("Not Available")
            else:
                hotel_price.append(str(tiles[i].find_all("div",{"class","priceBlock"})[0].find_all("div",{"class","price"})[0].text[2:]))
        else:
            hotel_price.append("Not Available")
    
    print len(hotel_names)," ",len(hotel_links)," ",len(hotel_price)    
            
    driver.find_element_by_xpath("//a[@data-page-number='"+str(j+1)+"']").click();
    WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, "//span[@data-page-number='"+str(j+1)+"']")))
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
    print j
    
time.sleep(3)

driver.find_element_by_xpath("//a[@id='global-nav-restaurants']").click();
time.sleep(3)
driver.find_element_by_xpath("//a[@id='global-nav-restaurants']").click();
print "Extracting Restaurants Data"
time.sleep(3)

src = driver.page_source
soup = BeautifulSoup(src, "lxml")

page_numbers = soup.find("div",{"class","pageNumbers"})
page_numbers.find_all('a')
last = page_numbers.find_all('a').pop()
size = int(last['data-page-number'])

restaurants_names = []
restaurants_links = []

size = 2 #remove this after demo

for j in range(1,size):
    
    tiles = soup.find_all("a",{"class":"property_title"})
     
    for i in range(0, len(tiles)):
        restaurants_names.append(tiles[i].text)    
        restaurants_links.append(tiles[i]['href'])
            
    driver.find_element_by_xpath("//a[@data-page-number='"+str(j+1)+"']").click();
    WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, "//span[@data-page-number='"+str(j+1)+"']")))
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
    print j

complete_hotel_data = []



Extracting Hotels Data
31   31   31
1
Extracting Restaurants Data
1


In [7]:
complete_hotel_data = []

print "Extracting Particular Hotel Data"
time.sleep(3)

for l in hotel_links:
    
    driver.get("https://www.tripadvisor.in"+l)
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
    
    name = soup.find("h1",{"id":"HEADING"})
    print name.text
    
    locality = soup.find_all("span",{"class":"locality"})[0].text
    if(len(soup.find_all("span",{"class":"extended-address"})) > 0):
        extended_address = soup.find_all("span",{"class":"extended-address"})[0].text
    else:
        extended_address = "Not Available"
        
    if(len(soup.find_all("span",{"class":"ib_price price"})) > 0):
        price = int(soup.find_all("span",{"class":"ib_price price"})[0].text[2:].replace(",",""))
    else :
        price = 999999
    #street_address = soup.find_all("span",{"class":"street-address"})[0].text
    
    #phone_number = str(soup.find_all("div",{"class":"blEntry phone"})[0].contents[1].text)
    
    outoff = str(soup.find_all("span",{"class":"header_popularity popIndexValidation"})[0].contents[1]
    )
    
    rank = str(soup.find_all("span",{"class":"header_popularity popIndexValidation"})[0].contents[0].text[1:])
    
    rating  = str(soup.find_all("span",{"class":"overallRating"})[0].text)
    
    excellent = soup.find_all("li",{"data-idx":"5"})[0].find("span",{"class":"row_count row_cell"}).text
    very_good = soup.find_all("li",{"data-idx":"4"})[0].find("span",{"class":"row_count row_cell"}).text
    average = soup.find_all("li",{"data-idx":"3"})[0].find("span",{"class":"row_count row_cell"}).text
    poor = soup.find_all("li",{"data-idx":"2"})[0].find("span",{"class":"row_count row_cell"}).text
    terrible = soup.find_all("li",{"data-idx":"1"})[0].find("span",{"class":"row_count row_cell"}).text
    
    features = []
    
    #for i in soup.find_all("div",{"class","detailsMid"})[0]:
    #   features.append(str(i.text))
    
    if( len(soup("div","pageNumbers")[0].find_all("span",{"class": "separator"})) >0 ):
        page_length = int(soup("div","pageNumbers")[0].find_all("span",{"class": "separator"})[0].nextSibling.text)
        page_length = 2 #remove this
    else:
        page_length = 1
        
    reviews=[]
    reviews_head =[]
    
    
    
    for i in range(0,page_length):    
    
        time.sleep(2)
        WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='taLnk ulBlueLinks']")))
        driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
        time.sleep(2)
        src = driver.page_source
        soup = BeautifulSoup(src, "lxml")
        
        for i in soup.find_all("span",{"class":"noQuotes"}):
            reviews.append(i.parent.parent.nextSibling.find_all("p",{"class":"partial_entry"})[0].text)
        
        reviews_head = reviews_head + soup.find_all("span",{"class","noQuotes"})
        time.sleep(1)
        WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='nav next taLnk ']")))    
        driver.find_element_by_xpath("//span[@class='nav next taLnk ']").click()
        print i
        
        
    particular_hotel_data = [name,locality,extended_address,rank,outoff,rating,excellent,very_good,average,poor,terrible,reviews,reviews_head,price]
    
    complete_hotel_data.append(particular_hotel_data)
    
    
complete_restaurants_data = []

print "Extracting Particular Restaurants Data"
time.sleep(3)

for l in restaurants_links:
    driver.get("https://www.tripadvisor.in"+l)
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
    
    name = soup.find("h1",{"id":"HEADING"})
    print name.text
    
    locality = soup.find_all("span",{"class":"locality"})[0].text
    
    if( len(soup.find_all("span",{"class":"extended-address"})) >0):
        extended_address = soup.find_all("span",{"class":"extended-address"})[0].text
    else:
        extended_address = "Not available"
        
    if len(soup.find_all("span",{"class":"street-address"})) > 0:    
        street_address = soup.find_all("span",{"class":"street-address"})[0].text
    else:
        street_address = "Not available"

    if(len(soup.find_all("div",{"class":"blEntry phone"})) > 0):
        
        phone_number = str(soup.find_all("div",{"class":"blEntry phone"})[0].contents[1].text)
    
    else:
        phone_number = "Not available"
    
    outoff = str(soup.find_all("span",{"class":"header_popularity popIndexValidation"})[0].contents[1]
    )
    
    rank = str(soup.find_all("span",{"class":"header_popularity popIndexValidation"})[0].contents[0].contents[0].text[1:])
    
    rating  = str(soup.find_all("span",{"class":"overallRating"})[0].text)
    
    excellent = soup.find_all("li",{"data-idx":"5"})[0].find("span",{"class":"row_count row_cell"}).text
    very_good = soup.find_all("li",{"data-idx":"4"})[0].find("span",{"class":"row_count row_cell"}).text
    average = soup.find_all("li",{"data-idx":"3"})[0].find("span",{"class":"row_count row_cell"}).text
    poor = soup.find_all("li",{"data-idx":"2"})[0].find("span",{"class":"row_count row_cell"}).text
    terrible = soup.find_all("li",{"data-idx":"1"})[0].find("span",{"class":"row_count row_cell"}).text
    
    features = []
    
    if(len(soup.find_all("div",{"class":"ui_icon restaurants"})) > 0):
        cuisines = str(soup.find_all("div",{"class":"ui_icon restaurants"})[0].nextSibling.text)
    else:
        cuisines = "Not Available"
    
    if( len(soup("div","pageNumbers")[0].find_all("span",{"class": "separator"})) >0 ):
        page_length = int(soup("div","pageNumbers")[0].find_all("span",{"class": "separator"})[0].nextSibling.text)
        page_length = 2 #remove this
    else:
        page_length = 1
        
    reviews=[]
    reviews_head =[]    
    
    for i in range(0,page_length-1):        
        time.sleep(2)
        WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='taLnk ulBlueLinks']")))        
        driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
        time.sleep(2)
        src = driver.page_source
        soup = BeautifulSoup(src, "lxml")
        
        for i in soup.find_all("span",{"class":"noQuotes"}):
            reviews.append(i.parent.parent.nextSibling.find_all("p",{"class":"partial_entry"})[0].text)
        
        reviews_head = reviews_head + soup.find_all("span",{"class","noQuotes"})
        time.sleep(1)
        WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='nav next taLnk ']")))    
        driver.find_element_by_xpath("//span[@class='nav next taLnk ']").click()
        print i
        
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
        
    for i in soup.find_all("span",{"class":"noQuotes"}):
        reviews.append(i.parent.parent.nextSibling.find_all("p",{"class":"partial_entry"})[0].text)
        reviews_head = reviews_head + soup.find_all("span",{"class","noQuotes"})

    
    particular_restaurants_data = [name,locality,extended_address,street_address,phone_number,rank,outoff,rating,excellent,very_good,average,poor,terrible,reviews,reviews_head,cuisines]
    
    complete_restaurants_data.append(particular_restaurants_data)


Extracting Particular Hotel Data
Hotel Fortune
<span class="noQuotes">it was excellent stay</span>
<span class="noQuotes">it was excellent stay</span>
Courtyard by Marriott Mumbai International Airport
<span class="noQuotes">Holidays</span>
<span class="noQuotes">Holidays</span>
Sun-n-Sand Hotel, Mumbai
<span class="noQuotes">good place to recover from jetlag before the onslaught of Mumbai</span>
<span class="noQuotes">good place to recover from jetlag before the onslaught of Mumbai</span>
The Westin Mumbai Garden City
<span class="noQuotes">Service attitude of restaurant staff</span>
<span class="noQuotes">Service attitude of restaurant staff</span>
The Lalit Mumbai
<span class="noQuotes">Supplier Days at The LaLit Mumbai</span>
<span class="noQuotes">Supplier Days at The LaLit Mumbai</span>
Chateau Windsor Hotel
<span class="noQuotes">Lovely Hotel</span>
<span class="noQuotes">Lovely Hotel</span>
The Orchid Mumbai Vile Parle
<span class="noQuotes">excellent service and gratitude</spa

In [8]:
complete_restaurants_data

[[<h1 class="heading_title" id="HEADING">Nawab Saheb</h1>,
  u'Mumbai 400087, ',
  u'#2 & 3B, Near Chinmayanand Ashram, Powai, Renaissance Mumbai Convention Centre Hotel',
  'Not available',
  '+91 22 6692 7558',
  '1',
  ' of 13,773 ',
  '5.0 ',
  u'83%',
  u'13%',
  u'2%',
  u'1%',
  u'1%',
  [u'My business trip has been one of firsts, two of which are this was my first ever visit to India, and my first ever visit to an Indian restaurant. I had the good fortune to visit restaurant Nawab Saheb, located in the Renaissance Mumbai Convention Centre Hotel.Everyone was very friendly and helpful explaining the menu. The server who took my order asked if I ever had Indian spices before, and he said he would advise the chef to modify the spice level for me. Later on, I had a chance to meet the chef, Kasim Khan, who came to my table to ask about how his preparation of the food was for me. I told him it was a delicious experience, and thanked him for personally talking with me. Later on, Chef K

In [9]:
complete_restaurants_data[0]


[<h1 class="heading_title" id="HEADING">Nawab Saheb</h1>,
 u'Mumbai 400087, ',
 u'#2 & 3B, Near Chinmayanand Ashram, Powai, Renaissance Mumbai Convention Centre Hotel',
 'Not available',
 '+91 22 6692 7558',
 '1',
 ' of 13,773 ',
 '5.0 ',
 u'83%',
 u'13%',
 u'2%',
 u'1%',
 u'1%',
 [u'My business trip has been one of firsts, two of which are this was my first ever visit to India, and my first ever visit to an Indian restaurant. I had the good fortune to visit restaurant Nawab Saheb, located in the Renaissance Mumbai Convention Centre Hotel.Everyone was very friendly and helpful explaining the menu. The server who took my order asked if I ever had Indian spices before, and he said he would advise the chef to modify the spice level for me. Later on, I had a chance to meet the chef, Kasim Khan, who came to my table to ask about how his preparation of the food was for me. I told him it was a delicious experience, and thanked him for personally talking with me. Later on, Chef Khan returned w

In [11]:
complete_hotel_data

[[<h1 class="heading_title" id="HEADING">Hotel Fortune</h1>,
  u'Mumbai 400002, ',
  u'Metro, Dhobi Talao',
  '337',
  ' of 456 ',
  '3.5 ',
  u'32%',
  u'19%',
  u'15%',
  u'15%',
  u'19%',
  [u'Rooms are not very good. Very conjugated. Service is not proper. Poor housekeeping in room. Take long time for food preparation. Near to marine lines but very noise area. not comfortable even in night 2 am.',
   u'We arrived here as part of a g adventures tour group - the lobby looked inviting. Unfortunately all of our rooms smelt overwhelmingly of damp and also all of our beds had damp bed sheets on them. When we confronted the hotel staff about it...More',
   u'////Hotel staff wary halp fully than indian food warry nice good service and rooms warry nice /////\nVery well maintained, staff is good, food is cooked properly, snack served is good compare to earlier party, but ladies hair serve to us this is very shocked...More',
   u'rooms are good. service is nice. all staff of this hotel very p

In [12]:
complete_hotel_data[0]

[<h1 class="heading_title" id="HEADING">Hotel Fortune</h1>,
 u'Mumbai 400002, ',
 u'Metro, Dhobi Talao',
 '337',
 ' of 456 ',
 '3.5 ',
 u'32%',
 u'19%',
 u'15%',
 u'15%',
 u'19%',
 [u'Rooms are not very good. Very conjugated. Service is not proper. Poor housekeeping in room. Take long time for food preparation. Near to marine lines but very noise area. not comfortable even in night 2 am.',
  u'We arrived here as part of a g adventures tour group - the lobby looked inviting. Unfortunately all of our rooms smelt overwhelmingly of damp and also all of our beds had damp bed sheets on them. When we confronted the hotel staff about it...More',
  u'////Hotel staff wary halp fully than indian food warry nice good service and rooms warry nice /////\nVery well maintained, staff is good, food is cooked properly, snack served is good compare to earlier party, but ladies hair serve to us this is very shocked...More',
  u'rooms are good. service is nice. all staff of this hotel very polite.house kee

In [19]:
import csv

#for i in complete_restaurants_data:
#    i[0] = i[0].text


for i in complete_restaurants_data:
    write_row = [x.encode('utf-8') for x in i[13]]
    h_name = str(i[0].encode("utf-8")).replace("/","")
    
    with open("restaurants_reviews/"+h_name+".csv","w") as review_head:
        csv_writer = csv.writer(review_head)
        csv_writer.writerow(write_row)
        
for i in complete_restaurants_data:
    reviews_write = list(set(i[14]))
    write_row = [x.text.encode('utf-8') for x in reviews_write]
    h_name = str(i[0].encode("utf-8")).replace("/","")
    
    with open("restaurants_reviews_head/"+h_name+".csv","w") as review_head:
        csv_writer = csv.writer(review_head)
        csv_writer.writerow(write_row)
        
for i in complete_hotel_data:
    write_row = [x.encode('utf-8') for x in i[11]]
    h_name = str(i[0].text.encode("utf-8")).replace("/","")
    
    with open("hotels_reviews/"+h_name+".csv","w") as review_head:
        csv_writer = csv.writer(review_head)
        csv_writer.writerow(write_row)
        
for i in complete_hotel_data:
    reviews_write = list(set(i[12]))
    write_row = [x.encode('utf-8') for x in reviews_write]
    h_name = str(i[0].text.encode("utf-8")).replace("/","")
    
    with open("hotels_reviews_head/"+h_name+".csv","w") as review_head:
        csv_writer = csv.writer(review_head)
        csv_writer.writerow(write_row)
        
#for i in complete_hotel_data:
#    i[0] = i[0].text
    

for i in complete_hotel_data:
    write_row = [x.encode('utf-8') for x in i[0:10]]
    h_name = str(i[0].text.encode("utf-8")).replace("/","")
    
    with open("hotels_details/"+h_name+".csv","w") as hotels_details:
        csv_writer = csv.writer(hotels_details)
        csv_writer.writerow(write_row)
        
for i in complete_restaurants_data:
    write_row = [x.encode('utf-8') for x in i[0:12]]
    h_name = str(i[0].encode("utf-8")).replace("/","")
    
    with open("restaurants_details/"+h_name+".csv","w") as restaurants_details:
        csv_writer = csv.writer(restaurants_details)
        csv_writer.writerow(write_row)
  
from textblob import TextBlob

import glob

for i in glob.glob("hotels_reviews/*.csv"):
    print i
    with open(i,"r") as sample:
        csv_reader = csv.reader(sample)
        sentiment = 0
        confidence = 0
        for line in csv_reader:
            for item in line:
                nlpblob = TextBlob(item.decode('utf-8'))
                sentiment += nlpblob.sentiment[0]
                confidence += nlpblob.sentiment[1]
            append_sentiment =  sentiment/len(line)
            append_confidence = confidence/len(line)
        
    with open(i,"a") as write_sample:
        csv_writer = csv.writer(write_sample)
        s = []
        t = []
        s.append(append_sentiment)
        t.append(append_confidence)
        csv_writer.writerow(s)
        csv_writer.writerow(t)
        
for i in glob.glob("restaurants_reviews/*.csv"):
    print i
    with open(i,"r") as sample:
        csv_reader = csv.reader(sample)
        sentiment = 0
        confidence = 0
        for line in csv_reader:
            for item in line:
                nlpblob = TextBlob(item.decode('utf-8'))
                sentiment += nlpblob.sentiment[0]
                confidence += nlpblob.sentiment[1]
            append_sentiment =  sentiment/len(line)
            append_confidence = confidence/len(line)
        
    with open(i,"a") as write_sample:
        csv_writer = csv.writer(write_sample)
        s = []
        t = []
        s.append(append_sentiment)
        t.append(append_confidence)
        csv_writer.writerow(s)
        csv_writer.writerow(t)
        
for i in glob.glob("hotels_reviews_head/*.csv"):
    print i
    with open(i,"r") as sample:
        csv_reader = csv.reader(sample)
        sentiment = 0
        confidence = 0
        for line in csv_reader:
            for item in line:
                nlpblob = TextBlob(item.decode('utf-8'))
                sentiment += nlpblob.sentiment[0]
                confidence += nlpblob.sentiment[1]
            append_sentiment =  sentiment/len(line)
            append_confidence = confidence/len(line)
        
    with open(i,"a") as write_sample:
        csv_writer = csv.writer(write_sample)
        s = []
        t = []
        s.append(append_sentiment)
        t.append(append_confidence)
        csv_writer.writerow(s)
        csv_writer.writerow(t)
        
for i in glob.glob("restaurants_reviews_head/*.csv"):
    print i
    with open(i,"r") as sample:
        csv_reader = csv.reader(sample)
        sentiment = 0
        confidence = 0
        for line in csv_reader:
            for item in line:
                nlpblob = TextBlob(item.decode('utf-8'))
                sentiment += nlpblob.sentiment[0]
                confidence += nlpblob.sentiment[1]
            append_sentiment =  sentiment/len(line)
            append_confidence = confidence/len(line)
        
    with open(i,"a") as write_sample:
        csv_writer = csv.writer(write_sample)
        s = []
        t = []
        s.append(append_sentiment)
        t.append(append_confidence)
        csv_writer.writerow(s)
        csv_writer.writerow(t)


hotels_reviews\Chateau Windsor Hotel.csv
hotels_reviews\Courtyard by Marriott Mumbai International Airport.csv
hotels_reviews\Four Seasons Hotel Mumbai.csv
hotels_reviews\Grand Hyatt Mumbai.csv
hotels_reviews\Grand Sarovar Premiere Mumbai.csv
hotels_reviews\Hilltop Hotel Mumbai.csv
hotels_reviews\Holiday Inn Mumbai International Airport.csv
hotels_reviews\Hotel Accord.csv
hotels_reviews\Hotel Fortune.csv
hotels_reviews\Hotel Sea Princess.csv
hotels_reviews\Hotel Travellers Inn.csv
hotels_reviews\Hyatt Regency Mumbai.csv
hotels_reviews\Ibis Mumbai Airport.csv
hotels_reviews\ITC Grand Central.csv
hotels_reviews\JW Marriott Mumbai Juhu.csv
hotels_reviews\Novotel Mumbai Juhu Beach.csv
hotels_reviews\Pali Beach Resort.csv
hotels_reviews\Renaissance Mumbai Convention Centre Hotel.csv
hotels_reviews\Residency Hotel Andheri.csv
hotels_reviews\Sahara Star Hotel.csv
hotels_reviews\Sofitel Mumbai BKC.csv
hotels_reviews\Sun-n-Sand Hotel, Mumbai.csv
hotels_reviews\Taj Lands End Mumbai.csv
hotels_re

In [2]:
import os
import sys
import time
import wget

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

print "Opening Trip Advisor"

driver = webdriver.Firefox(executable_path=r'C:\Program Files\geckodriver.exe')

Opening Trip Advisor


In [3]:
location = "Mumbai"
current = "Delhi"

In [4]:
driver.get("https://www.tripadvisor.in/CheapFlightsHome")

driver.find_element_by_xpath("//input[@tabindex='4']").clear()
driver.find_element_by_xpath("//input[@tabindex='4']").send_keys(current)
time.sleep(6)
driver.find_element_by_xpath("//input[@tabindex='4']").send_keys(Keys.ENTER)
driver.find_element_by_xpath("//input[@tabindex='5']").clear()
driver.find_element_by_xpath("//input[@tabindex='5']").send_keys(location)
time.sleep(6)
driver.find_element_by_xpath("//input[@tabindex='5']").send_keys(Keys.ENTER)
time.sleep(6)
driver.find_element_by_xpath("//span[@class='picker-label target']").click()
driver.find_element_by_xpath("//span[@data-date='2017-11-15']").click()
time.sleep(3)
driver.find_element_by_xpath("//span[@data-date='2017-11-25']").click()
time.sleep(6)
driver.find_element_by_xpath("//button[@tabindex='9']").click()
driver.switch_to_window(driver.window_handles[1])


In [5]:
WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, "//input[@id='awEmail']")))

driver.find_element_by_xpath("//input[@id='awEmail']").click();
driver.find_element_by_xpath("//input[@id='awEmail']").send_keys("hello@world.com")
driver.find_element_by_xpath("//div[@class='create-alert-btn']").click()

In [6]:
src = driver.page_source
soup = BeautifulSoup(src, "lxml")

airlines = soup.find_all("div",{"class":"mainFlightInfo"})
airlines_number = len(airlines)

airlines_name = []

for i in range(0,airlines_number):
    names = airlines[i].find_all("div",{"class","airlineName"})

    temp_name = []    
    
    for j in names :
        temp_name.append(str(j.text))
        
    airlines_name.append(temp_name)
    
    
airports = []

for i in range(0,airlines_number):    
    
    depart = soup.find_all("div",{"class":"segments"})[i].find_all("div",{"class":"departureDescription airportDescription"})
    arrival = soup.find_all("div",{"class":"segments"})[i].find_all("div",{"class":"arrivalDescription airportDescription"})

    depart_airports = []
    arrival_airports = []    
    
    for j in depart:
        depart_airports.append(str(j.text))
        
    for j in arrival:
        arrival_airports.append(str(j.text))
        
    current_airports = [depart_airports,arrival_airports]
    
    airports.append(current_airports)
    
airplanes_prices = []

prices  = soup.find_all("span",{"class","viewDealPrice"})

for i in prices:
    airplanes_prices.append(i.text[2:])
    
for i in range(0,len(airlines)):
    print airlines_name[i]
    print airports[i]
    print airplanes_prices[i]
    print "\n"


['IndiGo']
[['DEL 06:50', 'BOM 16:30'], ['BOM 13:45', 'DEL 23:35']]
6,354


['IndiGo']
[['DEL 22:35', 'BOM 16:30'], ['BOM 05:30+1', 'DEL 23:35']]
6,354


['IndiGo']
[['DEL 06:50', 'BOM 15:10'], ['BOM 13:45', 'DEL 22:45']]
6,354


['IndiGo']
[['DEL 22:35', 'BOM 15:10'], ['BOM 05:30+1', 'DEL 22:45']]
6,354


['IndiGo']
[['DEL 08:35', 'BOM 16:30'], ['BOM 16:20', 'DEL 23:35']]
6,354


['IndiGo']
[['DEL 08:35', 'BOM 15:10'], ['BOM 16:20', 'DEL 22:45']]
6,354


['IndiGo']
[['DEL 22:35', 'BOM 16:30'], ['BOM 07:10+1', 'DEL 23:35']]
6,354


['IndiGo']
[['DEL 07:40', 'BOM 16:30'], ['BOM 16:20', 'DEL 23:35']]
6,354


['IndiGo']
[['DEL 22:35', 'BOM 15:10'], ['BOM 07:10+1', 'DEL 22:45']]
6,354


['IndiGo']
[['DEL 07:40', 'BOM 15:10'], ['BOM 16:20', 'DEL 22:45']]
6,354


['IndiGo', 'SpiceJet']
[['DEL 06:50', 'BOM 07:20'], ['BOM 13:45', 'DEL 09:25']]
6,558


['IndiGo', 'SpiceJet']
[['DEL 22:35', 'BOM 07:20'], ['BOM 05:30+1', 'DEL 09:25']]
6,558


['IndiGo', 'GoAir']
[['DEL 06:50', 'BOM 22:30'], ['BOM

In [8]:
import csv
import glob

restaurants_reviews_rating = []
    
for i in glob.glob("restaurants_reviews/*.csv"):
    temp = []
    temp.append(i.split("\\")[1].split(".")[0])
    csv_path = i
    with open(csv_path, "rb") as f_obj:
        reader = csv.reader(f_obj)
        reader.next()
        temp.append(float(reader.next()[0]))
        temp.append(float(reader.next()[0]))
        restaurants_reviews_rating.append(temp)

hotels_reviews_rating = []
    
for i in glob.glob("hotels_reviews/*.csv"):
    temp = []
    temp.append(i.split("\\")[1].split(".")[0])
    csv_path = i
    with open(csv_path, "rb") as f_obj:
        reader = csv.reader(f_obj)
        reader.next()
        temp.append(float(reader.next()[0]))
        temp.append(float(reader.next()[0]))
        hotels_reviews_rating.append(temp)
        

hotels_reviews_head_rating = []
    
for i in glob.glob("hotels_reviews_head/*.csv"):
    temp = []
    temp.append(i.split("\\")[1].split(".")[0])
    csv_path = i
    with open(csv_path, "rb") as f_obj:
        reader = csv.reader(f_obj)
        reader.next()
        temp.append(float(reader.next()[0]))
        temp.append(float(reader.next()[0]))
        hotels_reviews_head_rating.append(temp)
        
restaurants_reviews_head_rating = []
    
for i in glob.glob("restaurants_reviews_head/*.csv"):
    temp = []
    temp.append(i.split("\\")[1].split(".")[0])
    csv_path = i
    with open(csv_path, "rb") as f_obj:
        reader = csv.reader(f_obj)
        reader.next()
        temp.append(float(reader.next()[0]))
        temp.append(float(reader.next()[0]))
        restaurants_reviews_head_rating.append(temp)


In [9]:
hotels_reviews_rating.sort(key = lambda hotels_reviews_rating : hotels_reviews_rating[1]*hotels_reviews_rating[2])
hotels_reviews_head_rating.sort(key = lambda hotels_reviews_head_rating : hotels_reviews_head_rating[1]*hotels_reviews_head_rating[2])
restaurants_reviews_rating.sort(key = lambda restaurants_reviews_rating : restaurants_reviews_rating[1]*restaurants_reviews_rating[2])
restaurants_reviews_head_rating.sort(key = lambda restaurants_reviews_head_rating : restaurants_reviews_head_rating[1]*restaurants_reviews_head_rating[2])


In [10]:
hotels_reviews_rating

[['Novotel Mumbai Juhu Beach', 0.027976482644339794, 0.6050492237920808],
 ['Chateau Windsor Hotel', 0.11117765567765567, 0.4249377289377289],
 ['Hotel Sea Princess', 0.15643997794898437, 0.4353282794241924],
 ['Sun-n-Sand Hotel, Mumbai', 0.20947976190476192, 0.5175004578754578],
 ['Ibis Mumbai Airport', 0.24954343915343918, 0.5297080952380953],
 ['Hotel Travellers Inn', 0.2427809801887927, 0.5483146645021645],
 ['The Orchid Mumbai Vile Parle', 0.29808333333333337, 0.4570619658119658],
 ['The Lalit Mumbai', 0.28342024324881465, 0.5316820861678005],
 ['Residency Hotel Andheri', 0.30391501261896, 0.5136946369906896],
 ['The Taj Mahal Palace, Mumbai', 0.3046591512438789, 0.5425524801740422],
 ['Four Seasons Hotel Mumbai', 0.25819910037878785, 0.6548717532467532],
 ['Hotel Accord', 0.3001952861952862, 0.5816917989417989],
 ['Grand Hyatt Mumbai', 0.3079175684269434, 0.5869820235320236],
 ['Hotel Fortune', 0.28591923076923076, 0.6400384615384616],
 ['Taj Lands End Mumbai', 0.3277661081047999

In [11]:
hotels_reviews_head_rating

[['Novotel Mumbai Juhu Beach', -0.14499999999999996, 0.17666666666666667],
 ['The Westin Mumbai Garden City', -0.02, 0.08],
 ['Sun-n-Sand Hotel, Mumbai', 0.0, 0.0],
 ['The Resort', 0.0, 0.05],
 ['Residency Hotel Andheri', 0.0009999999999999788, 0.33799999999999997],
 ['Four Seasons Hotel Mumbai', 0.0050000000000000044, 0.16499999999999998],
 ['The Taj Mahal Palace, Mumbai', 0.034999999999999996, 0.065],
 ['The Oberoi, Mumbai', 0.03666666666666667, 0.07],
 ['Trident, Bandra Kurla, Mumbai', 0.055499999999999994, 0.054000000000000006],
 ['Sofitel Mumbai BKC', 0.1, 0.03],
 ['Trident, Nariman Point', 0.04, 0.08],
 ['Holiday Inn Mumbai International Airport',
  0.03500000000000001,
  0.10500000000000001],
 ['Hotel Travellers Inn', 0.06666666666666667, 0.061111111111111116],
 ['Pali Beach Resort', 0.061, 0.10800000000000001],
 ['Courtyard by Marriott Mumbai International Airport', 0.075, 0.1],
 ['Ibis Mumbai Airport', 0.07600000000000001, 0.144],
 ['ITC Grand Central', 0.10500000000000001, 0.

In [12]:
restaurants_reviews_rating

[['Masala Library', 0.12179382885632886, 0.6082733431691765],
 ['Sea Lounge', 0.2294772297808012, 0.6004774187452759],
 ['Prego', 0.303077226076668, 0.519438517831375],
 ['The Peshwa Pavilion', 0.32292728747416255, 0.5187802323427324],
 ['Global Fusion Restaurant', 0.32729022892051557, 0.5698210481561046],
 ['Peshawri', 0.31398363095238097, 0.6209553571428572],
 ['Pondicherry Cafe', 0.32072623509531406, 0.622761403508772],
 ['Mahesh Lunch Home', 0.3388408206908208, 0.6017453147953148],
 ['Shree Thaker Bhojanalay', 0.33974603174603174, 0.600384126984127],
 ['Burma Burma Restaurant & Tea Room',
  0.35918925264550267,
  0.5893757936507937],
 ['Nawab Saheb', 0.3838216871597124, 0.5682788432039483],
 ["Hornby's Pavilion", 0.33973470888355345, 0.6569922703696864],
 ['B Bhagat Tarachand', 0.45795555555555556, 0.5232484126984127],
 ['Trishna', 0.38888891723356006, 0.6169471513605442],
 ['South of Vindhyas', 0.4125463864838864, 0.5853558201058202],
 ['Cafe Madras', 0.4357121212121212, 0.5750277

In [13]:
restaurants_reviews_head_rating

[['Seven Kitchens', 0.1, 0.1],
 ['Boulevard', 0.1757142857142857, 0.17714285714285716],
 ['Pondicherry Cafe', 0.09300000000000001, 0.375],
 ['Sea Lounge', 0.1271666666666667, 0.5051666666666667],
 ['Global Fusion Restaurant', 0.26749999999999996, 0.27749999999999997],
 ["Hornby's Pavilion", 0.16944444444444445, 0.49444444444444446],
 ["Romano's", 0.31657142857142856, 0.2646547619047619],
 ['Status', 0.27, 0.33999999999999997],
 ['By the Mekong', 0.32866666666666666, 0.3413333333333333],
 ['Trishna', 0.325, 0.41],
 ['Mahesh Lunch Home', 0.41, 0.4125],
 ['Masala Library', 0.3169444444444444, 0.5347222222222222],
 ['Prego', 0.38, 0.47000000000000003],
 ['Peshawri', 0.3793333333333333, 0.5023333333333333],
 ['Shree Thaker Bhojanalay', 0.36375, 0.545],
 ['Barbeque Nation', 0.385, 0.53],
 ['Mostly Grills', 0.4387348484848485, 0.4703333333333333],
 ['South of Vindhyas', 0.3453787878787879, 0.6100000000000001],
 ['Mirchi and Mime', 0.4199285714285715, 0.5172857142857141],
 ['Burma Burma Restau

In [28]:
print "Combo Packages"

print "Best Packages according to reviews"
for i in range(1,6):
    print "Hotel : "+hotels_reviews_rating[-i][0]+"\n"
    print "Restaurants : "
    for j in restaurants_reviews_rating[-1:-6:-1]:
        print j[0]
    print "\n"
    
print "Best Packages according to Review Keywords "
for i in range(1,6):
    print "Hotel : "+hotels_reviews_head_rating[-i][0]+"\n"
    print "Restaurants : "
    for j in restaurants_reviews_head_rating[-1:-6:-1]:
        print j[0]
    print "\n"

print "Recommended Flights for you "
for i in range(0,5):
    print airlines_name[i]
    print airports[i]
    print airplanes_prices[i]
    print "\n"

Combo Packages
Hotel : Courtyard by Marriott Mumbai International Airport

Restaurants : 
Boulevard
MoMo Cafe
Ark Bar
Seasonal Tastes
Lake View Cafe


Hotel : Trident, Nariman Point

Restaurants : 
Boulevard
MoMo Cafe
Ark Bar
Seasonal Tastes
Lake View Cafe


Hotel : Grand Sarovar Premiere Mumbai

Restaurants : 
Boulevard
MoMo Cafe
Ark Bar
Seasonal Tastes
Lake View Cafe


Hotel : The Westin Mumbai Garden City

Restaurants : 
Boulevard
MoMo Cafe
Ark Bar
Seasonal Tastes
Lake View Cafe


Hotel : JW Marriott Mumbai Juhu

Restaurants : 
Boulevard
MoMo Cafe
Ark Bar
Seasonal Tastes
Lake View Cafe


Hotel : The Leela Mumbai

Restaurants : 
Dum Pukht
JW Cafe
Seasonal Tastes
The Peshwa Pavilion
Lake View Cafe


Hotel : Renaissance Mumbai Convention Centre Hotel

Restaurants : 
Dum Pukht
JW Cafe
Seasonal Tastes
The Peshwa Pavilion
Lake View Cafe


Hotel : The Lalit Mumbai

Restaurants : 
Dum Pukht
JW Cafe
Seasonal Tastes
The Peshwa Pavilion
Lake View Cafe


Hotel : Grand Hyatt Mumbai

Restaurants 